# 1) Titolo e obiettivi
Lezione 38: Introduzione ai modelli generativi con un bigramma semplice.

---

## Mappa concettuale della lezione

```
MODELLI GENERATIVI - PANORAMICA
================================

MODELLI DISCRIMINATIVI          MODELLI GENERATIVI
==================              ==================
P(y|x) - "data x, che classe?"  P(x) o P(x|z) - "genera x"

Input ──► Modello ──► Label     Latent/Context ──► Modello ──► Output
                                        
Esempio:                        Esempio:
- Classificazione               - Generazione testo
- Regressione                   - Sintesi immagini
- Object detection              - Data augmentation


MODELLO AUTOREGRESSIVO (Bigram)
================================

    P(frase) = P(w₁) × P(w₂|w₁) × P(w₃|w₂) × ... × P(wₙ|wₙ₋₁)

    <s> ──► "apri" ──► "un" ──► "ticket" ──► </s>
         P(apri|<s>)  P(un|apri) P(ticket|un)  P(</s>|ticket)


GENERAZIONE CON TEMPERATURA
============================

    BASSA TEMPERATURA (T→0)         ALTA TEMPERATURA (T→∞)
    ========================         ========================
    Sceglie parola piu' probabile   Scelta quasi uniforme
    Output deterministico           Output molto casuale
    Ripetitivo ma coerente          Creativo ma incoerente
```

---

## Obiettivi didattici

| # | Obiettivo | Livello |
|---|-----------|---------|
| 1 | Distinguere modelli generativi da discriminativi | Concettuale |
| 2 | Implementare modello bigram con conteggi | Operativo |
| 3 | Generare testo campionando da distribuzioni | Pratico |
| 4 | Controllare creativita' con temperatura | Avanzato |
| 5 | Riconoscere limiti dei modelli n-gram | Critico |
| 6 | Preparare transizione verso modelli neurali | Prospettiva |

---

## Concetti chiave

> **Modello generativo**: modello che impara la distribuzione P(x) dei dati e puo' generare nuovi esempi campionando da essa.

> **Autoregressivo**: genera sequenze un token alla volta, condizionando ogni token sui precedenti: P(x₁, x₂, ..., xₙ) = ∏ P(xᵢ|x₁...xᵢ₋₁).

> **Temperatura**: parametro che scala i logits prima del softmax; T<1 sharpening (piu' deterministico), T>1 flattening (piu' casuale).

---

## Tassonomia modelli generativi

```
+------------------------+------------------------+
|   MODELLO              |   COMPLESSITA'         |
+------------------------+------------------------+
| Bigram                 | ★☆☆☆☆ (baseline)       |
| N-gram (n>2)           | ★★☆☆☆                  |
| RNN/LSTM               | ★★★☆☆                  |
| Transformer            | ★★★★☆                  |
| GPT/LLM                | ★★★★★                  |
+------------------------+------------------------+

Bigram: memoria di 1 token - questa lezione
N-gram: memoria di n-1 token - estensione diretta
RNN: memoria "teoricamente" infinita - vanishing gradient
Transformer: attenzione globale - stato dell'arte
```

---

## Cosa useremo
- Conteggi di bigrammi con `defaultdict`
- Campionamento con `np.random.choice`
- Controllo temperatura per diversita'
- Corpus di esempio (frasi assistenza clienti)

## Prerequisiti
- Tokenizzazione base (Lezione 30)
- Probabilita' condizionata
- Concetto di likelihood


# 2) Teoria concettuale
- Modello generativo: stima P(token_t | token_{t-1}) da un corpus.
- Smoothing/temperatura: controlla diversita' vs ripetizione.
- Limiti: nessuna memoria lunga; non gestisce sintassi complessa.


# 3) Schema mentale / mappa decisionale
1. Tokenizza il corpus.
2. Conta bigrammi -> probabilita' condizionate.
3. Genera testo scegliendo parola successiva in base a P(.|prev) con temperatura.
4. Valuta qualitativamente, aggiungi dati per migliorare copertura.


# 4) Sezione dimostrativa
Demo: costruzione modello bigram su corpus assistenza clienti, generazione e osservazioni.


In [ ]:
# Setup
import numpy as np
from collections import defaultdict, Counter
np.random.seed(42)


In [ ]:
# Corpus su assistenza clienti
corpus = [
    "apri un ticket per assistenza tecnica",
    "richiedi rimborso per ordine difettoso",
    "verifica lo stato della spedizione",
    "contatta il supporto per informazioni",
    "chiudi il ticket risolto"
]


In [ ]:
# Tokenizzazione semplice e conteggio bigrammi
start = '<s>'
end = '</s>'

bigrams = defaultdict(Counter)
for doc in corpus:
    tokens = [start] + doc.split() + [end]
    for a, b in zip(tokens, tokens[1:]):
        bigrams[a][b] += 1

# Calcolo probabilita' condizionate
probs = {a: {b: c/sum(cnt.values()) for b, c in cnt.items()} for a, cnt in bigrams.items()}
print("Esempio probs per '<s>':", probs[start])
assert start in probs


In [ ]:
# Generazione autoregressiva con temperatura

def sample_next(distrib, temperature=1.0):
    words = list(distrib.keys())
    logits = np.log(np.array(list(distrib.values())) + 1e-8) / temperature
    probs_temp = np.exp(logits) / np.sum(np.exp(logits))
    return np.random.choice(words, p=probs_temp)

def generate(max_len=15, temperature=1.0):
    word = start
    out = []
    for _ in range(max_len):
        next_w = sample_next(probs[word], temperature)
        if next_w == end:
            break
        out.append(next_w)
        word = next_w
    return ' '.join(out)

print("Generazione (temp=1.0):", generate())
print("Generazione (temp=0.5):", generate(temperature=0.5))


### Osservazioni
- Con temperatura bassa il testo e' piu' ripetitivo ma coerente; alta temperatura aumenta diversita' e rumore.
- Corpus piccolo limita la copertura e porta a frasi brevi.


# 5) Esercizi svolti (step-by-step)
## Esercizio 38.1 - Probabilita' di una frase


In [ ]:
# Esercizio 38.1: calcola P("apri un ticket")
phrase = [start, 'apri', 'un', 'ticket', end]
prob_phrase = 1.0
for a, b in zip(phrase, phrase[1:]):
    prob_phrase *= probs.get(a, {}).get(b, 1e-8)
print(f"Probabilita' frase: {prob_phrase:.6f}")


## Esercizio 38.2 - Temperatura diversa


In [ ]:
# Esercizio 38.2: genera con temperature diverse
for t in [0.3, 0.7, 1.2]:
    print(f"Temp={t}: {generate(temperature=t)}")


## Esercizio 38.3 - Aggiungere dati


In [ ]:
# Esercizio 38.3: aggiungi un documento e rigenera
corpus_extra = corpus + ["richiedi assistenza per rimborso spedizione"]
# ricalcolo rapido
bigrams2 = defaultdict(Counter)
for doc in corpus_extra:
    toks = [start] + doc.split() + [end]
    for a,b in zip(toks, toks[1:]):
        bigrams2[a][b] += 1
probs2 = {a:{b:c/sum(cnt.values()) for b,c in cnt.items()} for a,cnt in bigrams2.items()}
probs = probs2  # aggiorna il modello
print("Generazione dopo aggiornamento:", generate())


# 6) Conclusione operativa - Bignami Modelli Generativi

---

## I 5 Take-Home Messages

| # | Concetto | Perche' conta |
|---|----------|---------------|
| 1 | **Generativo vs Discriminativo** | Generativo modella P(x), discriminativo P(y|x) |
| 2 | **Autoregressivo = token by token** | Ogni token dipende dai precedenti |
| 3 | **Bigram = memoria corta** | Solo 1 token di contesto, limiti evidenti |
| 4 | **Temperatura controlla creativita'** | Trade-off coerenza vs diversita' |
| 5 | **Smoothing per token mai visti** | Evita probabilita' zero su bigrammi nuovi |

---

## Pipeline generazione testo

```
TRAINING                           INFERENCE
========                           =========

Corpus ──► Tokenize ──► Count      Seed ──► Sample ──► Append ──► Loop
                          |                    ↑            |
                          v                    |            |
                      P(w|prev)                └────────────┘
                          |                    until </s>
                          v
                   Normalize probs


TEMPERATURA (formula)
=====================

    P_temp(w) = exp(log P(w) / T) / Σ exp(log P(w') / T)

    T = 1.0  →  Probabilita' originali
    T < 1.0  →  Sharpening (piu' deterministico)
    T > 1.0  →  Flattening (piu' casuale)
```

---

## Confronto temperature

| Temperatura | Effetto | Output tipico |
|-------------|---------|---------------|
| 0.1 | Quasi greedy | "apri un ticket per assistenza" (ripetitivo) |
| 0.5 | Bilanciato | "richiedi rimborso per ordine" (coerente) |
| 1.0 | Originale | "apri un ticket difettoso" (variabile) |
| 2.0 | Casuale | "per ticket rimborso un" (incoerente) |

---

## Template modello bigram

```python
from collections import defaultdict
import numpy as np

class BigramModel:
    def __init__(self):
        self.bigrams = defaultdict(lambda: defaultdict(int))
        self.vocab = set()
    
    def fit(self, corpus):
        """Corpus: lista di frasi tokenizzate"""
        for sentence in corpus:
            tokens = ['<s>'] + sentence.split() + ['</s>']
            for i in range(len(tokens) - 1):
                self.bigrams[tokens[i]][tokens[i+1]] += 1
                self.vocab.update(tokens)
    
    def get_probs(self, prev_token, temperature=1.0):
        """Ritorna distribuzione P(next|prev) con temperatura"""
        counts = self.bigrams[prev_token]
        if not counts:
            return None  # Token mai visto
        words = list(counts.keys())
        freqs = np.array([counts[w] for w in words], dtype=float)
        # Applica temperatura
        log_probs = np.log(freqs + 1e-10) / temperature
        probs = np.exp(log_probs) / np.sum(np.exp(log_probs))
        return dict(zip(words, probs))
    
    def generate(self, max_len=20, temperature=1.0):
        """Genera frase autoregressivamente"""
        tokens = ['<s>']
        for _ in range(max_len):
            probs = self.get_probs(tokens[-1], temperature)
            if probs is None:
                break
            next_token = np.random.choice(list(probs.keys()), p=list(probs.values()))
            if next_token == '</s>':
                break
            tokens.append(next_token)
        return ' '.join(tokens[1:])
    
    def sentence_probability(self, sentence):
        """Calcola P(frase) = prodotto P(w_i|w_{i-1})"""
        tokens = ['<s>'] + sentence.split() + ['</s>']
        prob = 1.0
        for i in range(len(tokens) - 1):
            probs = self.get_probs(tokens[i])
            if probs and tokens[i+1] in probs:
                prob *= probs[tokens[i+1]]
            else:
                return 0.0  # Bigram mai visto
        return prob

# Uso
model = BigramModel()
model.fit(corpus)
print(model.generate(temperature=0.7))
print(f"P(frase): {model.sentence_probability('apri un ticket'):.6f}")
```

---

## Errori comuni e soluzioni

| Errore | Conseguenza | Soluzione |
|--------|-------------|-----------|
| No marker <s>/<\/s> | Non sa dove iniziare/finire | Aggiungere sempre i marker |
| Temperatura = 0 | Divisione per zero | Usare T >= 0.01 |
| Corpus piccolo | Molti bigram mancanti | Add-k smoothing o piu' dati |
| Loop infiniti | Frase non termina | Max length + check </s> |

---

## Metodi e concetti chiave

| Elemento | Ruolo |
|----------|-------|
| `defaultdict(int)` | Conteggi bigram |
| Probabilita' condizionata | P(next\|prev) = count(prev,next) / count(prev) |
| Temperatura | Controlla entropia della distribuzione |
| `np.random.choice` | Campionamento da distribuzione |
| Log-probability | Per evitare underflow su frasi lunghe |


# 7) Checklist di fine lezione
- [ ] Ho tokenizzato e aggiunto marker <s>/<\s> per inizio/fine.
- [ ] Ho calcolato correttamente le probabilita' condizionate bigram.
- [ ] Ho testato generazione con temperature diverse.
- [ ] Ho aggiunto dati o smoothing se i bigram mancavano.
- [ ] Ho valutato qualitativamente le frasi generate.

Glossario
- Bigram: coppia di parole consecutive.
- Temperatura: parametro che calibra la casualita' nel campionamento.
- Autoregressivo: genera token successivi basandosi sugli precedenti.
- Smoothing: tecnica per gestire probabilita' di eventi non visti.


# 8) Changelog didattico

| Versione | Data | Modifiche |
|----------|------|-----------|
| 1.0 | 2024-01-XX | Struttura iniziale 8 sezioni |
| 2.0 | 2024-12-XX | Espansione completa Modelli Generativi |
| 2.1 | - | Confronto generativo vs discriminativo ASCII |
| 2.2 | - | Tassonomia modelli (bigram → GPT) |
| 2.3 | - | Formula temperatura con effetti visualizzati |
| 2.4 | - | Classe BigramModel completa con metodi |
| 2.5 | - | Tabella confronto temperature con esempi output |
| 2.6 | - | Calcolo probabilita' di frase |

---

## Note di versione

**v2.0 - Espansione didattica completa**
- Posizionamento generativo vs discriminativo chiaro
- Modello autoregressivo spiegato con formula prodotto
- Temperatura come parametro di controllo creativita'
- Classe template riutilizzabile con tutti i metodi
- Preparazione per modelli piu' complessi (RNN, Transformer)

**Dipendenze didattiche**
- Richiede: Lezione 30 (tokenizzazione), probabilita' base
- Prepara: Comprensione di LLM e generazione neurale
